test tra i vari algoritmi di preprocess

In [13]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

path_image_giorno = "/prova_giorno.png"
path_image_notte = "/prova_notte.png"
image_giorno = cv2.imread(path_image_giorno)
image_notte = cv2.imread(path_image_notte)
print(image_giorno)

None


In [10]:
!dir

 Il volume nell'unit� C non ha etichetta.
 Numero di serie del volume: 56AA-DDF7

 Directory di c:\Users\danie\OneDrive\Documenti\GitHub\projectSmartVeychularSystem\test

21/01/2025  16:42    <DIR>          .
21/01/2025  16:40    <DIR>          ..
21/01/2025  15:37             1.899 Precoces_confronto.ipynb
11/01/2025  20:42           786.432 prova_giorno.png
11/01/2025  23:40           567.190 prova_notte.png
21/01/2025  14:48             3.285 testAngolo.ipynb
21/01/2025  15:23           124.093 testRadar.ipynb
               5 File      1.482.899 byte
               2 Directory  21.109.309.440 byte disponibili


In [12]:
if image_giorno is None or image_notte is None:
    print("Errore nel caricamento delle immagini. Controlla i percorsi.")
else:
    # Configura il layout per mostrare le immagini
    plt.figure(figsize=(12, 6))

    # Immagine Giorno
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(image_giorno, cv2.COLOR_BGR2RGB))
    plt.title("Immagine Giorno")
    plt.axis("off")

    # Immagine Notte
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(image_notte, cv2.COLOR_BGR2RGB))
    plt.title("Immagine Notte")
    plt.axis("off")

    # Mostra le immagini
    plt.tight_layout()
    plt.show()

Errore nel caricamento delle immagini. Controlla i percorsi.


preprocess soglia globale

In [ ]:
def preprocess_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)
    kernel = np.ones((5, 5), np.uint8)
    mask_cleaned = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
    return mask_cleaned

In [ ]:
img_matches = preprocess_image(image)

cv2.imshow('Matches', img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()

preprocess con maschera in HSV

In [ ]:
def preprocess_image(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 50, 255])
    binary_mask = cv2.inRange(hsv_image, lower_white, upper_white)
    kernel = np.ones((5, 5), np.uint8)
    mask_cleaned = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
    return mask_cleaned

preprocess con soglia locale

In [ ]:
def preprocess_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    adaptive_thresh = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                           cv2.THRESH_BINARY, 11, 2)
    kernel = np.ones((3, 13), np.uint8)
    opened_mask = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_OPEN, kernel)
    contours, _ = cv2.findContours(opened_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    final_mask = np.zeros_like(opened_mask)
    for contour in contours:
        if cv2.contourArea(contour) >= 1500:
            cv2.drawContours(final_mask, [contour], -1, (255), thickness=cv2.FILLED)
    return final_mask